In [1]:
%pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 1.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
import mysql.connector

class MariaDBURID:
    def __init__(self, host, port, user, password, database):
        self.connection = mysql.connector.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=database
        )
        self.cursor = self.connection.cursor()

    def get_database_list(self):
        self.cursor.execute("SHOW DATABASES")
        database_list = [row[0] for row in self.cursor.fetchall()]
        return database_list

    def get_table_list(self, database):
        self.cursor.execute(f"SHOW TABLES FROM {database}")
        table_list = [row[0] for row in self.cursor.fetchall()]
        return table_list

    def get_table_schema(self, database, table):
        self.cursor.execute(f"DESCRIBE {database}.{table}")
        table_schema = self.cursor.fetchall()
        return table_schema

    def get_user_list(self):
        self.cursor.execute("SELECT user FROM mysql.user")
        user_list = [row[0] for row in self.cursor.fetchall()]
        return user_list

    def execute_query(self, query):
        self.cursor.execute(query)
        result = self.cursor.fetchall()
        return result

    def insert_data(self, table, columns, values):
        columns_str = ', '.join(columns)
        values_str = ', '.join(["%s"] * len(values))
        query = f"INSERT INTO {table} ({columns_str}) VALUES ({values_str})"
        self.cursor.execute(query, values)
        self.connection.commit()

    def insert_data_ignore_duplicates(self, table, columns, values):
        columns_str = ', '.join(columns)
        values_str = ', '.join(["%s"] * len(values))
        query = f"INSERT IGNORE INTO {table} ({columns_str}) VALUES ({values_str})"
        self.cursor.execute(query, values)
        self.connection.commit()

    def insert_data_with_conditions(self, table, columns, values, cond, cond_values):
        columns_str = ', '.join(columns)
        values_str = ', '.join(["%s"] * len(values))
        query = f"INSERT INTO {table} ({columns_str}) SELECT {values_str} WHERE {cond}"
        self.cursor.execute(query, values + cond_values)
        self.connection.commit()

    def update_data(self, table, columns, values, condition):
        columns_str = ', '.join([f"{col} = %s" for col in columns])
        query = f"UPDATE {table} SET {columns_str} WHERE {condition}"
        self.cursor.execute(query, values)
        self.connection.commit()

    def insert_data_on_duplicate_key_update(self, table, columns, values, update_columns, update_values, cond):
        columns_str = ', '.join(columns)
        values_str = ', '.join(["%s"] * len(values))
        update_str = ', '.join([f"{column} = %s" for column in update_columns])
        query = f"INSERT INTO {table} ({columns_str}) VALUES ({values_str}) ON DUPLICATE KEY UPDATE {update_str} {cond}"
        self.cursor.execute(query, values + update_values)
        self.connection.commit()

    def delete_data(self, table, condition):
        query = f"DELETE FROM {table} WHERE {condition}"
        self.cursor.execute(query)
        self.connection.commit()

    def close_connection(self):
        self.cursor.close()
        self.connection.close()

In [10]:
# 建立 MariaDBURID 對象
# db = MariaDBURID(host='localhost', port=3306, user='developer', password='password', database='JudgmentsDB')
db = MariaDBURID(host='3.115.42.166', port=3306, user='tstudent02', password='Scsb@2024', database='tstudent02db')

In [4]:
import pandas as pd
df = pd.read_csv('demo/dataset2_no_JFULL.csv')
print(df.shape)

(4012, 10)


In [6]:
for cols,row in df.iterrows():
    if isinstance(row['JID'], str) and row['JID'].strip():
        db.insert_data_ignore_duplicates('Judgments'
                                     , columns=['JID', 'JTITLE', 'JCHAR', 'JTYPE', 'JDATE', 'JURL', 'JDESP']
                                     , values=[row['JID']
                                               , row['JTITLE'] if isinstance(row['JTITLE'], str) else None
                                               , row['JCHAR'] if isinstance(row['JCHAR'], str) else None
                                               , row['JTYPE'] if isinstance(row['JTYPE'], str) else None
                                               , row['JDATE'] if isinstance(row['JDATE'], str) else None
                                               , row['JURL'] if isinstance(row['JDATE'], str) else None
                                               , row['JSUMMARY'] if isinstance(row['JDATE'], str) else None
                                               ])
    if isinstance(row['JOBJECT'], str) and row['JOBJECT'].strip():
        db.insert_data_with_conditions('Searchable'
                                     , columns=['JID', 'JSUBJECT']
                                     , values=[row['JID'], row['JOBJECT']]
                                     , cond=f"NOT EXISTS (SELECT 1 FROM Searchable WHERE JID = %s AND JSUBJECT = %s);"
                                     , cond_values=[row['JID'], row['JOBJECT']])
        

In [12]:
ret = db.execute_query("SELECT * FROM Searchable WHERE JSUBJECT='長鴻營造'")
print(ret[0])

[(1, 'TPSV,112,台上,2737,20240530,1', '長鴻營造', None, None, None), (2, 'TPHV,112,建上更一,7,20240326,1', '長鴻營造', None, None, None), (3, 'TPHV,112,上,646,20240123,1', '長鴻營造', None, None, None), (4, 'TPHV,109,建上,98,20240123,3', '長鴻營造', None, None, None), (5, 'TPDM,111,金重訴緝,1,20240118,3', '長鴻營造', None, None, None), (6, 'KSDV,106,建,34,20231226,2', '長鴻營造', None, None, None), (7, 'KSHV,111,建上更一,6,20231213,3', '長鴻營造', None, None, None), (8, 'TPHV,112,重上,474,20231128,1', '長鴻營造', None, None, None), (9, 'TPDV,112,國,30,20231121,1', '長鴻營造', None, None, None), (10, 'TPSV,112,台上,1876,20230913,1', '長鴻營造', None, None, None), (11, 'TPHV,111,建上,43,20230830,1', '長鴻營造', None, None, None), (12, 'KSDV,108,建,92,20230829,1', '長鴻營造', None, None, None), (13, 'CTDV,112,簡上,25,20230531,1', '長鴻營造', None, None, None), (14, 'TPHV,110,重上,618,20230502,2', '長鴻營造', None, None, None), (15, 'TPSV,112,台上,61,20230426,1', '長鴻營造', None, None, None), (16, 'PCDV,111,重訴,693,20230331,1', '長鴻營造', None, None, None), (17, 'TPHV,110,重上,249,202

In [20]:
print(ret[0])

(1, 'TPSV,112,台上,2737,20240530,1', '長鴻營造', None, None, None)


In [8]:
# 關閉連接
db.close_connection()